In [1]:
!pwd

/mnt/d/research/software/mosdef_cp2kmcpy/mosdef_cp2kmcpy/examples/GEMC_h2_h2o


In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import mbuild as mb
import foyer
from foyer import Forcefield

import mbuild.formats.charmm_writer as mf_charmm

import mosdef_cp2k_writer


import parmed as pmd
from constrainmol import ConstrainedMolecule
import unyt as u

from mosdef_cp2kmcpy.mc import MC
import setter
warnings.filterwarnings("ignore", category=DeprecationWarning)
 

In [2]:

hydrogen_res_name = 'H2'
FF_file_hydrogen = 'h2_ff.xml'
hydrogen_FF = Forcefield(forcefield_files=FF_file_hydrogen)
hydrogen_mb = mb.load('[HH]', smiles=True)
for particle in hydrogen_mb.particles():
    particle.name = "HH"
    particle.element = "H"
    
    
hydrogen_mb.name = hydrogen_res_name

hydrogen_typed=hydrogen_FF.apply(hydrogen_mb)



constrain_mol = ConstrainedMolecule(hydrogen_typed)
constrain_mol.solve()
hydrogen_optimized_mb=mb.clone(hydrogen_mb)
hydrogen_optimized_mb.xyz=constrain_mol.xyz/10


/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/foyer/forcefield.py:448: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/foyer/forcefield.py:460: UserWarning: No force field name found in force field XML file.
  warnings.warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:687: UserWarning: No unitcell detected for pybel.Molecule [H][H]	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.1015,-0.0038,-0.0091), 0 bonds, id: 140504471747984>" is element: "H"
  warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to

In [3]:
mb.__version__

'0.10.8'

In [4]:

water_res_name = 'H2O'
FF_file_water = 'water_tip3p.xml'
water_FF = Forcefield(forcefield_files=FF_file_water)
water_mb = mb.load('O', smiles=True)

for particle in water_mb.particles():
    
    if particle.name == "O":
        particle.element = "O"
    else:
        particle.element = "H"
        
water_mb.name = water_res_name

water_typed=water_FF.apply(water_mb)



constrain_mol = ConstrainedMolecule(water_typed)
constrain_mol.solve()
water_optimized_mb=mb.clone(water_mb)
water_optimized_mb.xyz=constrain_mol.xyz/10

/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:687: UserWarning: No unitcell detected for pybel.Molecule O	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


In [5]:
#random box to be used in mosdef_charmm_writer
hydrogen_box_bias  = mb.fill_box(compound=[hydrogen_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

water_box_bias  = mb.fill_box(compound=[water_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

mixed_box  = mb.fill_box(compound=[water_box_bias, hydrogen_optimized_mb ],
                             n_compounds=[1, 1] ,
                            box=[1.0, 1.0, 1.0] )



FF_Dict = { water_optimized_mb.name:FF_file_water, hydrogen_optimized_mb.name:FF_file_hydrogen }

residues_List = [water_optimized_mb.name, hydrogen_optimized_mb.name ]




#hydrogen_box_bias.save("hydrogen_bias_coord.xyz",overwrite=True)
#nitrogen_box_bias.save("nitrogen_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(mixed_box ,
                            'overall_bias',
                            FF_filename ="overall_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )



/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.0939,-0.0038,-0.0091), 0 bonds, id: 140504451349808>" is element: "H"
  warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.0465, 0.0268, 0.7991), 0 bonds, id: 140505726287056>" is element: "H"
  warn(


Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'H2O': 'water_tip3p.xml', 'H2': 'h2_ff.xml'}, residues = ['H2O', 'H2']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'H2O': 'water_tip3p.xml', 'H2': 'h2_ff.xml'}
coulomb14scale from compound = {'H2O': 0.5, 'H2': 0.5}
lj14scale from compound = {'H2O': 0.5, 'H2': 0.5}
unique_types = ['H_H2', 'h_H2O', 'o_H2O']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {2: 'o_H2O', 1: 'h_H2O', 0: 'H_H2'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'H2O': 'water_tip3p.xml', 'H2': 'h2_ff.xml'}, residues = ['H2O', 'H2']
******************************

No urey bradley terms det

/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.0465, 0.0268, 0.7991), 0 bonds, id: 140505725587568>" is element: "H"
  warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


In [6]:
FF_Dict = {hydrogen_optimized_mb.name:FF_file_hydrogen }

residues_List = [hydrogen_optimized_mb.name ]




#hydrogen_box_bias.save("hydrogen_bias_coord.xyz",overwrite=True)
#nitrogen_box_bias.save("nitrogen_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(hydrogen_box_bias ,
                            'hydrogen_bias',
                            FF_filename ="hydrogen_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )




Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'H2': 'h2_ff.xml'}, residues = ['H2']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'H2': 'h2_ff.xml'}
coulomb14scale from compound = {'H2': 0.5}
lj14scale from compound = {'H2': 0.5}
unique_types = ['H_H2']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {0: 'H_H2'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'H2': 'h2_ff.xml'}, residues = ['H2']
******************************

No urey bradley terms detected
******************************

write_charmm_pdb file is running
write_charmm_pdb: residues == ['H2']
fix_residue = None
fix_residue_in_box = None
bead_

/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.0166, 0.0275, 0.7999), 0 bonds, id: 140505726195696>" is element: "H"
  warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.0166, 0.0275, 0.7999), 0 bonds, id: 140504471168480>" is element: "H"
  warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


In [7]:
FF_Dict = { water_optimized_mb.name:FF_file_water }

residues_List = [water_optimized_mb.name ]




#hydrogen_box_bias.save("hydrogen_bias_coord.xyz",overwrite=True)
#nitrogen_box_bias.save("nitrogen_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(water_box_bias ,
                            'water_bias',
                            FF_filename ="water_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )



Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'H2O': 'water_tip3p.xml'}, residues = ['H2O']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'H2O': 'water_tip3p.xml'}
coulomb14scale from compound = {'H2O': 0.5}
lj14scale from compound = {'H2O': 0.5}
unique_types = ['h_H2O', 'o_H2O']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {1: 'o_H2O', 0: 'h_H2O'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'H2O': 'water_tip3p.xml'}, residues = ['H2O']
******************************

No urey bradley terms detected
******************************

write_charmm_pdb file is running
write_charmm_pdb: residues == ['H2O']
f

In [8]:
molecule_list=[water_optimized_mb, hydrogen_optimized_mb]
box=mb.box.Box(lengths=[1,1,1])
#Comment why pressure is needed


In [9]:
q=MC(molecule_list=molecule_list,n_box=2,n_molecules_each_box=[[2,2],[2,2]], box_list=[box,box],cutoff=200,functional='PBE',
     basis_set={'H':'DZVP-MOLOPT-GTH', 'O':'DZVP-MOLOPT-GTH','HH':'DZVP-MOLOPT-GTH'}, periodicity=['XYZ']*2,ensemble='GEMC_NPT',seed=1,project_name='water_NPT_GEMC',restart='FALSE',pressure=1*u.bar, use_atom_name_as_symbol = False, run_type="equilibration")#, initial_coordinate_filename = ["box1_xyz.xyz", "box2_xyz.xyz"])

In [10]:
q.mc_initialization()

scf_tolerance not specified, set as 1e-6
basis_set_filename not defined, set as BASIS_MOLOPT
potential_filename not specified, set as GTH_POTENTIALS
n_steps not specified, set as 1000
iuptrans not specified, set as 10
iupvolume not specified, set as 10
n_ff_moves not specified, set as 8
nswapmoves not specified, set as 640, will be ignore if n_box<2
output trajectory format set as XYZ
input_filename not specified, set as ['GEMC_NPT_box1.inp', 'GEMC_NPT_box2.inp']
output_filename not specified, set as water_NPT_GEMC_mc_output.out
temperature not defined, set as 298 K
You can change default settings in setter.mc_files


In [11]:

q.topology_filename=['water_bias.psf', 'hydrogen_bias.psf']

# move_probabilities=[pmavbmc,pmcltrans,pmhmc,pmswap,pmtraion,pmtrans,pmvolume]
#volume moves = PMVOLUME
# swap moves = PMSWAP - PMVOLUME
# AVBMC moves = PMAVBMC - PMSWAP - PMVOLUME
# an “inner” move = 1.0 - (PMAVBMC + PMSWAP + PMVOLUME)
#conformational changes = “inner” move percentage × PMTRAION
# molecular translation = “inner” move percentage × (PMTRANS - PMTRAION)
# molecular rotation = “inner” move percentage × (1.0 - PMTRANS - PMTRAION)

q.move_probabilities=[0,0.0,0,0.4,0.5,0.75,0.05]



# mol_probabilities=[[PMAVBMC_MOL,PMSWAP_MOL , PMTRAION_MOL, PMTRANS_MOL,PMROT_MOL],[]]
q.mol_probabilities=[[[1, 1],[1, 1],[1, 1],[1, 1],[1, 1]],[[1, 1],[1, 1],[1, 1],[1, 1],[1, 1]]]



#avbmc probabilities=[[AVBMC_ATOM,AVBMC_RMIN,AVBMC_RMAX,PBIAS],[]]
q.avbmc_probabilities=[[[1, 1],[1, 1],[1, 1],[1, 1]],[[1, 1],[1, 1],[1, 1],[1, 1]]]

q.charmm_potential_file='overall_charmm_bias.inp'

In [12]:
q.iuptrans

10

In [20]:
q.input_filename

['GEMC_NPT_box1.inp', 'GEMC_NPT_box2.inp']

In [13]:

setter.mc_files(q)

MC initial structure saved as ['water_NPT_GEMC_box1_initial.xyz', 'water_NPT_GEMC_box2_initial.xyz']
IUPTRANS is 10
This is ETA [0, 0]
MC input file saved as GEMC_NPT_box1.inp
IUPTRANS is 10
This is ETA [0.0, 0]
MC input file saved as GEMC_NPT_box2.inp
IUPTRANS is 100
This is ETA [0.0]


/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.3822, 0.7415, 0.1669), 0 bonds, id: 140419875125856>" is element: "H"
  warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.3822, 0.7415, 0.1669), 0 bonds, id: 140419875139056>" is element: "H"
  warn(


In [14]:
!cat GEMC_NPT_box1.inp

&FORCE_EVAL
  METHOD        QUICKSTEP
  STRESS_TENSOR        ANALYTICAL
  &DFT        
    BASIS_SET_FILE_NAME        BASIS_MOLOPT
    CHARGE        0
    MULTIPLICITY        1
    POTENTIAL_FILE_NAME        GTH_POTENTIALS
    &MGRID        
      CUTOFF      200
      NGRIDS      4
      REL_CUTOFF      60
    &END MGRID        
    &QS        
      EPS_DEFAULT       1e-08
      EXTRAPOLATION       ASPC
      METHOD       GPW
    &END QS        
    &POISSON        
      PERIODIC       XYZ
      &EWALD        
      &END EWALD        
    &END POISSON        
    &PRINT         
      &E_DENSITY_CUBE       OFF
      &END E_DENSITY_CUBE       
    &END PRINT        
    &SCF        
      EPS_SCF       1e-06
      MAX_SCF       20
      SCF_GUESS       RESTART
      &OT       .TRUE. 
        MINIMIZER       DIIS
        PRECONDITIONER       FULL_SINGLE_INVERSE
      &END OT        
      &OUTER_SCF       .TRUE. 
        EPS_SCF       1e-06
        MAX_SCF       10
      &END OUTER_SC

In [21]:
!cat box1_xyz.xyz

cat: box1_xyz.xyz: No such file or directory


In [22]:
!cat dlt.xyz

4
dlt.xyz - created by mBuild
H    1.000000    1.000000    1.000000
O    2.000000    2.000000    2.000000
HH    1.000000    3.000000    2.000000
HH    2.000000    3.000000    4.000000
